In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from corpus_reading import read_dump

chords_tsv = 'data/chord_list.tsv'
notes_tsv = 'data/note_list.tsv'
measures_tsv = 'data/measure_list.tsv'
files_tsv = 'data/file_list.tsv'

chords_df = read_dump(chords_tsv)
notes_df = read_dump(notes_tsv, index_col=[0,1,2])
measures_df = read_dump(measures_tsv)
files_df = read_dump(files_tsv, index_col=0)

# Bugfixes
measures_df.loc[(685, 487), 'next'][0] = 488

In [ ]:
files_df

In [ ]:
chords_df

In [ ]:
measures_df

In [ ]:
notes_df

In [ ]:
import corpus_utils
import importlib
importlib.reload(corpus_utils)

measures_removed_df = corpus_utils.remove_repeats(measures_df)
offset_mc, offset_beat = corpus_utils.get_offsets(notes_df, measures_removed_df)
offset_df = notes_df.assign(offset_mc=offset_mc, offset_beat=offset_beat)
merged_notes_df = corpus_utils.merge_ties(offset_df, measures=measures_removed_df)

In [ ]:
import corpus_utils
import harmonic_inference_data as hid
import importlib
importlib.reload(hid)
importlib.reload(corpus_utils)

dataset = hid.MusicScoreDataset(chords_df=chords_df, notes_df=merged_notes_df,
                                measures_df=measures_removed_df, files_df=files_df,
                                merge_ties=False)

In [ ]:
# np.concatenate((np.array([midi_pitch_norm]), tpc_one_hot, octave_one_hot,
#                               np.array([onset_level]), np.array([offset_level]),
#                               np.array([float(onset)]), np.array([float(offset)]), np.array([float(duration)]),
#                               np.array([is_lowest])))
notes = np.vstack([data['notes'] for data in dataset])

In [ ]:
from tqdm import tqdm

for data in tqdm(dataset):
    pass

In [ ]:
print(dataset[1001])

In [ ]:
print(chords_df.iloc[1001])

In [ ]:
def next2sequence(nxt):
    """Turns a pd.Series of lists into a sequence of elements."""
    i = nxt.index[0]
    last_i = nxt.index[-1]
    acc = [i]
    nxt = nxt.to_dict()
    flag = 0
    nxt_list = nxt[i]
    l = len(nxt_list)
    while i <= last_i:
        if i == last_i:
            if flag or l == 0:
                break
            elif l == 1: # last mc has repeat sign
                i = nxt_list[0]
                flag = 1
            else:
                raise NotImplementedError
        elif l == 1:
            i = nxt_list[0]
        elif l == 2:
            i = nxt_list[flag]
            flag = int(not flag)
        else:
            raise NotImplementedError("More than two voltas.")
        acc.append(i)
        nxt_list = nxt[i]
        l = len(nxt_list)
    return acc